## GRU使用案例

In [3]:
import torch
import torch.nn as nn

# 假设有一个用户点击了 5 个商品，每个商品的 embedding 是 16 维
seq_len = 5
embedding_dim = 16
hidden_dim = 16

# 模拟用户点击序列：shape (batch_size=1, seq_len=5, embedding_dim=16)
click_seq = torch.randn(1, seq_len, embedding_dim)

# 定义 GRU 层
gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, batch_first=True)

# 前向传播
output, hidden = gru(click_seq)

# 输出结果
print("GRU 所有时间步的输出 shape:", output.shape)   # [1, 5, 16]
print(output)
print("GRU 最后一个时间步的 hidden state shape:", hidden.shape)  # [1, 1, 16]
print(hidden)

# 取出最后的兴趣表示
interest_vec = hidden.squeeze(0).squeeze(0)  # shape: [16]
print("最终兴趣表示（兴趣演化结果）:", interest_vec.shape)

GRU 所有时间步的输出 shape: torch.Size([1, 5, 16])
tensor([[[ 3.7238e-02, -1.5648e-01, -4.3226e-02, -9.4178e-02, -2.6412e-01,
          -9.3574e-02,  2.6362e-01,  8.5652e-02, -3.3842e-02,  3.7506e-01,
          -1.0965e-01, -2.4402e-01, -1.4860e-01, -5.8236e-02,  4.6927e-01,
          -4.2615e-02],
         [-3.9359e-01, -6.9379e-02, -2.2097e-01, -5.1626e-02,  9.4833e-02,
           1.2797e-01, -3.5772e-02, -4.6342e-02, -4.5102e-02, -3.1505e-01,
           1.5444e-01, -4.6119e-01, -6.1264e-02, -3.2367e-01,  3.1226e-01,
           3.0412e-01],
         [-1.0793e-02, -3.6166e-02, -4.3215e-01, -2.3884e-01,  2.6443e-01,
           5.6652e-02,  2.3405e-01,  2.9078e-01,  4.4021e-01, -1.0093e-01,
           5.1605e-03, -8.9444e-02,  2.3829e-01, -2.4321e-01, -2.5449e-01,
           1.2529e-01],
         [ 2.1393e-01,  1.0465e-01, -3.1762e-01, -4.2221e-01,  8.3622e-02,
           3.7851e-01, -3.0766e-01,  4.1295e-01,  5.1438e-01,  2.9169e-01,
           3.3147e-02, -4.1506e-01,  2.6753e-01, -3.8061e-01

## DIEN代码

In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F

# ===================== 定义 embedding 层 =====================
shop_embedding = nn.Embedding(1000, 8)       # 店铺ID的embedding，词表大小1000，维度8
product_embedding = nn.Embedding(500, 16)   # 商品ID的embedding，词表大小500，维度16
category_embedding = nn.Embedding(100, 32)  # 商品类别的embedding，词表大小100，维度32

age_embedding = nn.Embedding(10, 8)       # 年龄类别的embedding，词表大小10，维度8
gender_embedding = nn.Embedding(3, 4)     # 性别类别的embedding，词表大小3，维度4
city_embedding = nn.Embedding(1000, 16)   # 城市类别的embedding，词表大小1000，维度16

# ===================== DIN 模型定义 =====================
class DIN(nn.Module):
    def __init__(self, shop_emb_dim=8, product_emb_dim=16):
        super(DIN, self).__init__()

        self.shop_emb_dim = shop_emb_dim          # 店铺embedding维度
        self.product_emb_dim = product_emb_dim    # 商品embedding维度

        # 定义针对店铺的注意力网络：输入是目标与历史embedding拼接后的向量，输出一个权重
        self.shop_attention_fc = nn.Sequential(
            nn.Linear(shop_emb_dim * 2, 64),      # 输入维度为2倍embedding维度（目标和历史拼接）
            nn.ReLU(),
            nn.Linear(64, 1)                      # 输出为一个标量权重
        )
        # 定义针对商品的注意力网络
        self.product_attention_fc = nn.Sequential(
            nn.Linear(product_emb_dim * 2, 64),
            nn.ReLU(),
            nn.Linear(64, 1)
        )

        # 新增GRU层，用于捕捉兴趣随时间的演变（序列特征建模）
        self.shop_gru = nn.GRU(input_size=shop_emb_dim, hidden_size=shop_emb_dim, batch_first=True)
        self.product_gru = nn.GRU(input_size=product_emb_dim, hidden_size=product_emb_dim, batch_first=True)

        # 定义多层感知机（MLP）：
        # 输入是年龄(8) + 性别(4) + 城市(16) + 活跃天数(1) + 店铺兴趣向量(8) + 商品兴趣向量(16)
        # + 店铺GRU演化向量(8) + 商品GRU演化向量(16)
        self.mlp = nn.Sequential(
            nn.Linear(8 + 4 + 16 + 1 + shop_emb_dim + product_emb_dim + shop_emb_dim + product_emb_dim, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 1)    # 最终输出一个标量预测值
        )

    # 定义注意力计算过程
    def attention(self, target_vec, history_vecs, attention_fc):
        target_expand = target_vec.unsqueeze(0).expand_as(history_vecs)  # 扩展目标向量，使其与历史序列长度相同，形状(seq_len, emb_dim)
        att_input = torch.cat([target_expand, history_vecs], dim=-1)     # 拼接目标与历史向量，形成(seq_len, 2*emb_dim)
        att_weights = attention_fc(att_input)                            # 计算每个历史行为的注意力权重，形状(seq_len, 1)
        att_weights = F.softmax(att_weights, dim=0)                      # 对历史行为按序列维度softmax归一化权重
        weighted_history = att_weights * history_vecs                    # 用注意力权重加权历史行为向量
        return weighted_history.sum(dim=0)                               # 聚合成单个兴趣向量，形状(emb_dim,)

    # 前向传播过程
    def forward(self, age_vec, gender_vec, city_vec, active_days,
                target_shop_vec, history_shop_vec,
                target_product_vec, history_product_vec):

        # 计算店铺兴趣向量（注意力加权历史店铺行为）
        shop_interest_vec = self.attention(target_shop_vec, history_shop_vec, self.shop_attention_fc)
        # 计算商品兴趣向量（注意力加权历史商品行为）
        product_interest_vec = self.attention(target_product_vec, history_product_vec, self.product_attention_fc)

        # GRU捕捉用户兴趣随时间的演变过程
        history_shop_vec = history_shop_vec.unsqueeze(0)    # 增加batch维度，变成(1, seq_len, shop_emb_dim)
        _, shop_evolved = self.shop_gru(history_shop_vec)   # 输出最后一个时间步的隐状态，形状(1, 1, shop_emb_dim)
        shop_evolved = shop_evolved.squeeze(0).squeeze(0)   # 去除多余维度，变成(shop_emb_dim,)

        history_product_vec = history_product_vec.unsqueeze(0)  # 同理处理商品历史序列
        _, product_evolved = self.product_gru(history_product_vec)
        product_evolved = product_evolved.squeeze(0).squeeze(0) # (product_emb_dim,)

        # 拼接所有特征形成最终输入向量
        features = torch.cat([
            age_vec, gender_vec, city_vec, active_days,
            shop_interest_vec, product_interest_vec,
            shop_evolved, product_evolved
        ], dim=-1)

        output = self.mlp(features)          # 通过MLP输出预测值
        return torch.sigmoid(output)         # 使用sigmoid映射到概率区间[0,1]


# ===================== 数据准备 =====================
# 用户特征示例
age = torch.tensor([3])                         # 年龄类别
gender = torch.tensor([1])                      # 性别类别
city = torch.tensor([25])                       # 城市类别
shop_category = torch.randint(0, 1000, (100,)) # 用户历史浏览的100个店铺ID序列
product_category = torch.randint(0, 500, (100,))  # 用户历史浏览的100个商品ID序列
target_shop = torch.tensor([100])               # 当前目标店铺ID
target_product = torch.tensor([150])            # 当前目标商品ID

active_days = torch.tensor([120.0])             # 用户活跃天数（未归一化）
normalized_active_days = active_days / 365      # 将活跃天数归一化到0~1区间

# 获取对应的embedding向量
age_vec = age_embedding(age)                     # (1, 8)
gender_vec = gender_embedding(gender)            # (1, 4)
city_vec = city_embedding(city)                   # (1, 16)
target_shop_vec = shop_embedding(target_shop)    # (1, 8)
history_shop_vec = shop_embedding(shop_category) # (100, 8)
target_product_vec = product_embedding(target_product)   # (1, 16)
history_product_vec = product_embedding(product_category) # (100, 16)

# 扩展活跃天数维度，方便与其他特征拼接
normalized_active_days_exp = normalized_active_days.expand(1, 1)  # (1, 1)

# ===================== 模型预测 =====================
model = DIN()     # 初始化模型
prediction = model(
    age_vec.squeeze(0),                     # 去除batch维度，变成(8,)
    gender_vec.squeeze(0),                  # (4,)
    city_vec.squeeze(0),                    # (16,)
    normalized_active_days_exp.squeeze(0), # (1,)
    target_shop_vec.squeeze(0),             # (8,)
    history_shop_vec,                       # (seq_len=100, 8)
    target_product_vec.squeeze(0),         # (16,)
    history_product_vec                     # (seq_len=100, 16)
)

print(f"预测的点击概率: {prediction.item():.4f}")   # 输出最终点击概率

预测的点击概率: 0.5349
